In [134]:
module Ev3dev

import Base: read, parse, write

parse(::Type{ASCIIString}, s::ASCIIString) = s
parse(::Type{Vector{ASCIIString}}, s::ASCIIString) = split(s)

immutable Brick
    root_path::AbstractString
    
    Brick(root_path::AbstractString="/") = new(root_path)
end


@inline novalidate(x) = true

abstract Attribute{T}

immutable ReadOnly{T} <: Attribute{T}
    name::Symbol
end
readable(::ReadOnly) = true
writable(::ReadOnly) = false

immutable WriteOnly{T} <: Attribute{T}
    name::Symbol
    validator::Function
end
readable(::WriteOnly) = false
writable(::WriteOnly) = true

immutable ReadWrite{T} <: Attribute{T}
    name::Symbol
    validator::Function
end
readable(::ReadWrite) = true
writable(::ReadWrite) = true

immutable WrappedStream{T}
    stream::IOStream
    validator::Function
end

function WrappedStream{T}(port_path::AbstractString, attribute::Attribute{T})
    path = joinpath(port_path, string(attribute.name))
    isfile(path) || error("file not found: $path")
    readable(attribute) && (isreadable(path) || error("read access for attribute $(attribute.name) was requested, but file is not readable"))
    writable(attribute) && (iswritable(path) || error("write access for attribute $(attribute.name) was requested, but file is not writable"))

    stream = open(path, readable(attribute), writable(attribute), false, writable(attribute), false)
    WrappedStream{T}(stream, attribute.validator)
end

function WrappedStream{T}(port_path::AbstractString, attribute::ReadOnly{T})
    path = joinpath(port_path, string(attribute.name))
    isfile(path) || error("file not found: $path")
    isreadable(path) || error("read access for attribute $(attribute.name) was requested, but file is not readable")
    stream = open(path, true, false, false, false, false)
    WrappedStream{T}(stream, novalidate)
end

function read{T}(stream::WrappedStream{T})
    seekstart(stream.stream)
    parse(T, readchomp(stream.stream))
end

function write{T}(stream::WrappedStream{T}, val::T)
    @assert stream.validator(val)
    seekstart(stream.stream)
    write(stream.stream, string(val))
    flush(stream.stream)
end

write{T}(stream::WrappedStream{T}, val) = write(stream, convert(T, val))

immutable Port
    brick::Brick
    path::AbstractString
    streams::Dict{Symbol, WrappedStream}
end

function Port(brick::Brick, path::AbstractString, 
    attributes::AbstractArray{Attribute})
    
    streams = Dict{Symbol, WrappedStream}()
    for attribute in attributes
        streams[attribute.name] = WrappedStream(joinpath(brick.root_path, path), attribute)
    end
    Port(brick, path, streams)
end

read(port::Port, name::Symbol) = read(port.streams[name])
write(port::Port, name::Symbol, value) = write(port.streams[name], value)

abstract Device
read(device::Device, name::Symbol) = read(device.port, name)
write(device::Device, name::Symbol, value) = write(device.port, name, value)

function find_device{T <: Device}(::Type{T}, brick::Brick, address::AbstractString, path_to_search::AbstractString)
    path = joinpath(brick.root_path, path_to_search)
    for dir in readdir(path)
        device_address = open(joinpath(path, dir, "address")) do f
            readchomp(f)
        end
        if device_address == address
            return T(brick, dir)
        end
    end
    nothing
end

macro make_device(name, path, attributes)
    quote type $(esc(name)) <: Device
            port::Port
        end
        
        function $(esc(name))(brick::Brick, identifier::AbstractString) 
            $(esc(name))(Port(brick, joinpath($(path), identifier), $(attributes)))
        end
        
        function $(esc(:find_device))(::Type{$(name)}, brick::Brick, address::AbstractString)
            find_device($(name), brick, address, $(path))
        end
        
        function make_accessors()
            for attribute in $(attributes)
                if readable(attribute)
                    expr = Expr(:(=), Expr(:call, attribute.name, Expr(:(::), :dev, $(esc(name)))), 
                    Expr(:call, :read, :dev, QuoteNode(attribute.name)))
                    eval(expr)
                end
                if writable(attribute)
                    expr = Expr(:(=), Expr(:call, symbol(attribute.name, :!), Expr(:(::), :dev, $(esc(name))), :value), 
                    Expr(:call, :write, :dev, QuoteNode(attribute.name), :value))
                    eval(expr)
                end
            end
        end
        make_accessors()
    end
end

@make_device Motor "sys/class/tacho-motor" [
    ReadOnly{ASCIIString}(:address),
    ReadOnly{Vector{ASCIIString}}(:commands),
    ReadOnly{ASCIIString}(:driver_name),
    ReadOnly{Int}(:position),
    ReadOnly{Int}(:count_per_rot),
    ReadOnly{Int}(:duty_cycle),
    ReadWrite{Int}(:duty_cycle_sp, x -> x > 0),
    ReadWrite{Int}(:speed_sp, novalidate),
    ReadWrite{Int}(:position_sp, novalidate),
    ReadWrite{ASCIIString}(:polarity, x -> x in Set(("normal", "inversed"))),
    WriteOnly{ASCIIString}(:command, novalidate),
    ReadWrite{ASCIIString}(:stop_action, novalidate)
    ]

@make_device Sensor "sys/class/lego-sensor" [
    ReadOnly{ASCIIString}(:address),
    ReadOnly{Vector{ASCIIString}}(:commands),
    ReadOnly{ASCIIString}(:driver_name),
    ReadOnly{Int}(:decimals),
    ReadOnly{Int}(:value0),
    ReadOnly{Int}(:value1),
    ReadOnly{Int}(:value2),
    ReadOnly{Int}(:value3),
    ReadOnly{Int}(:value4),
    ReadOnly{Int}(:value5),
    ReadOnly{Int}(:value6),
    ReadOnly{Int}(:value7),
    ReadOnly{Vector{ASCIIString}}(:modes),
    ReadOnly{ASCIIString}(:bin_data),
    ReadOnly{ASCIIString}(:bin_data_format),
    ReadWrite{Int}(:poll_ms, x -> x > 0)
    ]

end

import Ev3dev

In [135]:
brick = Ev3dev.Brick("/home/pi/ev3")

motor = Ev3dev.find_device(Ev3dev.Motor, brick, "outA")
ultrasound = Ev3dev.find_device(Ev3dev.Sensor, brick, "in1")

Ev3dev.Sensor(Ev3dev.Port(Ev3dev.Brick("/home/pi/ev3"),"sys/class/lego-sensor/sensor0",Dict{Symbol,Ev3dev.WrappedStream{T}}(:value6=>Ev3dev.WrappedStream{Int32}(IOStream(<file /home/pi/ev3/sys/class/lego-sensor/sensor0/value6>),Ev3dev.novalidate),:value5=>Ev3dev.WrappedStream{Int32}(IOStream(<file /home/pi/ev3/sys/class/lego-sensor/sensor0/value5>),Ev3dev.novalidate),:value4=>Ev3dev.WrappedStream{Int32}(IOStream(<file /home/pi/ev3/sys/class/lego-sensor/sensor0/value4>),Ev3dev.novalidate),:value3=>Ev3dev.WrappedStream{Int32}(IOStream(<file /home/pi/ev3/sys/class/lego-sensor/sensor0/value3>),Ev3dev.novalidate),:bin_data_format=>Ev3dev.WrappedStream{ASCIIString}(IOStream(<file /home/pi/ev3/sys/class/lego-sensor/sensor0/bin_data_format>),Ev3dev.novalidate),:value0=>Ev3dev.WrappedStream{Int32}(IOStream(<file /home/pi/ev3/sys/class/lego-sensor/sensor0/value0>),Ev3dev.novalidate),:driver_name=>Ev3dev.WrappedStream{ASCIIString}(IOStream(<file /home/pi/ev3/sys/class/lego-sensor/sensor0/driver_n

In [140]:
Ev3dev.value0(ultrasound)

569

In [141]:
Ev3dev.driver_name(ultrasound)

"lego-ev3-us"

In [115]:
read(motor, :commands)

7-element Array{SubString{ASCIIString},1}:
 "run-forever"   
 "run-to-abs-pos"
 "run-to-rel-pos"
 "run-timed"     
 "run-direct"    
 "stop"          
 "reset"         

In [116]:
Ev3dev.commands(motor)

7-element Array{SubString{ASCIIString},1}:
 "run-forever"   
 "run-to-abs-pos"
 "run-to-rel-pos"
 "run-timed"     
 "run-direct"    
 "stop"          
 "reset"         

In [117]:
write(motor, :speed_sp, 500)

IOStream(<file /home/pi/ev3/sys/class/tacho-motor/motor0/speed_sp>)

In [118]:
Ev3dev.command!(motor, "run-forever")

IOStream(<file /home/pi/ev3/sys/class/tacho-motor/motor0/command>)

In [119]:
Ev3dev.command!(motor, "stop")

IOStream(<file /home/pi/ev3/sys/class/tacho-motor/motor0/command>)

In [122]:
Ev3dev.polarity(motor)

"normal"